In [ ]:
import requests
import datetime
from transformers import pipeline

In [ ]:
NEWS_API_URL = "https://newsapi.org/v2/everything"
FX_API_URL = "https://api.exchangerate-api.com/v4/latest/USD"

In [ ]:
# دریافت اخبار
def get_news():
    params = {
        'q': 'USD',
        'apiKey': '8689e34ee2b64fabaff04f67f5f01269',
        'language': 'en',
        'sortBy': 'relevancy',
        'from': (datetime.datetime.now() - datetime.timedelta(days=1)).strftime('%Y-%m-%d')
    }
    response = requests.get(NEWS_API_URL, params=params)
    news_data = response.json()
    return news_data['articles']

In [ ]:
get_news()[0]

{'source': {'id': None, 'name': 'Mental Floss'},
 'author': 'Lorna Wallace',
 'title': 'The 10 Biggest Concerts in History',
 'description': 'Rod Stewart, Madonna, and the Rolling Stones had some of the largest concert crowds in history.',
 'url': 'https://www.mentalfloss.com/posts/biggest-concerts-in-history',
 'urlToImage': None,
 'publishedAt': '2024-08-28T12:01:00Z',
 'content': 'It can sometimes seem as though millions of people are cramming into concert venues to see the biggest artists in the world. Although no ticketed gig has ever actually reached attendance numbers in t… [+7826 chars]'}

In [ ]:
# دریافت قیمت دلار
def get_fx_rate():
    response = requests.get(FX_API_URL)
    fx_data = response.json()
    return fx_data['rates']['USD']

In [ ]:
get_fx_rate()

1

In [ ]:
# پردازش اخبار
def process_news(articles):
    nlp = pipeline("sentiment-analysis")
    sentiments = [nlp(article['description'])[0] for article in articles]
    return sentiments

In [ ]:
process_news(get_news())

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'POSITIVE', 'score': 0.99972003698349},
 {'label': 'POSITIVE', 'score': 0.9993665814399719},
 {'label': 'NEGATIVE', 'score': 0.9913151860237122},
 {'label': 'NEGATIVE', 'score': 0.9959931373596191},
 {'label': 'POSITIVE', 'score': 0.9836640357971191},
 {'label': 'POSITIVE', 'score': 0.9884679913520813},
 {'label': 'POSITIVE', 'score': 0.9995638728141785},
 {'label': 'NEGATIVE', 'score': 0.9885118007659912},
 {'label': 'POSITIVE', 'score': 0.9453237056732178},
 {'label': 'POSITIVE', 'score': 0.9957804679870605},
 {'label': 'POSITIVE', 'score': 0.9964945912361145},
 {'label': 'NEGATIVE', 'score': 0.999365508556366},
 {'label': 'POSITIVE', 'score': 0.992827832698822},
 {'label': 'POSITIVE', 'score': 0.9997422099113464},
 {'label': 'POSITIVE', 'score': 0.9996380805969238},
 {'label': 'NEGATIVE', 'score': 0.9980949759483337},
 {'label': 'POSITIVE', 'score': 0.9991950392723083},
 {'label': 'POSITIVE', 'score': 0.994114100933075},
 {'label': 'NEGATIVE', 'score': 0.8829333186149597}

In [ ]:
# پیشبینی قیمت دلار
def predict_fx_rate(news_sentiments, current_fx_rate):
    positive_news = sum(1 for sentiment in news_sentiments if sentiment['label'] == 'POSITIVE')
    negative_news = sum(1 for sentiment in news_sentiments if sentiment['label'] == 'NEGATIVE')
    prediction = current_fx_rate + (positive_news - negative_news) * 0.01  # تغییر قیمت بر اساس تعداد اخبار مثبت و منفی
    return prediction

In [ ]:
predict_fx_rate(process_news(get_news()), get_fx_rate())

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


1.12

In [ ]:
!pip install openai==0.28

In [ ]:
import openai

openai.api_key = ''

def generate_forecast_text(prediction):
    response = openai.Completion.create(
      engine="text-davinci-003",
      prompt=f"The predicted exchange rate for USD is {prediction:.2f}. This is based on the latest news and market trends.",
      max_tokens=100
    )
    return response.choices[0].text.strip()


In [ ]:
generate_forecast_text(predict_fx_rate(process_news(get_news()), get_fx_rate()))

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


APIRemovedInV1: 

You tried to access openai.Completion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [ ]:
# from flask import Flask, render_template

# app = Flask(__name__)

# @app.route('/')
# def index():
#     news_articles = get_news()
#     current_fx_rate = get_fx_rate()
#     news_sentiments = process_news(news_articles)
#     prediction = predict_fx_rate(news_sentiments, current_fx_rate)
#     forecast_text = generate_forecast_text(prediction)
#     return render_template('index.html', current_fx_rate=current_fx_rate, forecast_text=forecast_text)

# if __name__ == "__main__":
#     app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
from google.colab import drive
drive.mount('/content/drive')